In [1]:
ver = 2

In [2]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import transformers

from sklearn.metrics import accuracy_score

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

tf.random.set_seed(0)
np.random.seed(0)

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train_data.csv')
test_origin = pd.read_csv(data_path+'test_data.csv')
topic_dict_origin = pd.read_csv(data_path+'topic_dict.csv')
sample_submission_origin = pd.read_csv(data_path+'sample_submission.csv')

In [4]:
train = train_origin.copy()
test = test_origin.copy()
topic_dict = topic_dict_origin.copy()
sample_submission = sample_submission_origin.copy()

In [5]:
tokenizer = transformers.BertTokenizerFast.from_pretrained(
    'kykim/bert-kor-base',
    cache_dir = '../tokenizer/kykim/bert-kor-base',
    do_lower_case = False
)

In [6]:
data_config = {}
data_config['max_length'] = 30

In [7]:
def cleaning_text(text):
    text = re.sub('[^a-z가-힣]', ' ', text.lower())
    text = re.sub('[\s]+', ' ', text)
    return text

In [8]:
X_train = train['title'].apply(cleaning_text)
X_train = X_train.values.tolist()

X_train_encoded = tokenizer(
    X_train,
    padding = 'max_length',
    truncation = True,
    max_length = data_config['max_length'],
    return_tensors = 'tf'
)

In [9]:
y_train = train['topic_idx'].values

In [10]:
bert_model = transformers.TFBertForSequenceClassification.from_pretrained(
    'kykim/bert-kor-base', 
    cache_dir = '../model/kykim/bert-kor-base/cache/',
    output_hidden_states=False,
    output_attentions=False,
    use_cache = False,
    num_labels = 7
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def set_model(learning_rate=3e-5, print_summary=False):

    input_ids = Input(batch_shape=(None, data_config['max_length']), dtype=tf.int32, name='input_ids')
    input_masks = Input(batch_shape=(None, data_config['max_length']), dtype=tf.int32, name='attention_masks')
    bert_outputs = bert_model([input_ids, input_masks])['logits']
    dropout_0 = Dropout(0.2, name='dropout_0')(bert_outputs)
#     dense_0 = Dense(512, activation='relu', name='dense_0')(dropout_0)
#     dropout_1 = Dropout(0.1, name='dropout_1')(dense_0)
#     dense_1 = Dense(256, activation='relu', name='dense_1')(dropout_1)
#     dropout_2 = Dropout(0.1, name='dropout_2')(dense_1)
#     dense_2 = Dense(128, activation='relu', name='dense_2')(dropout_2)
#     dropout_3 = Dropout(0.1, name='dropout_3')(dense_2)
#     dense_3 = Dense(32, activation='relu', name='dense_3')(dropout_3)
#     dropout_4 = Dropout(0.1, name='dropout_4')(dense_3)
#     outputs = Dense(7, activation='softmax', name='outputs')(dropout_4)
    bn_0 = BatchNormalization()(dropout_0)
    outputs = Dense(7, activation='softmax', name='outputs')(bn_0)

    model = Model(
        inputs = [input_ids, input_masks],
        outputs = outputs,
        name = 'Bert_Classification'
    )

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = optimizer, 
        metrics = ['accuracy']
    )

    if print_summary:
        model.summary(line_length=150)
    
    return model

In [12]:
model = set_model(learning_rate=9e-5)
model.layers[2].trainable = False
for i in range(len(model.layers[2].weights)):
    model.layers[2].weights[i]._trainable = False
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f78289e8110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f78289e8110> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "Bert_Classification"
__________________________________________________________________________

In [13]:
history = model.fit(
    x = [X_train_encoded['input_ids'][:5000], X_train_encoded['attention_mask'][:5000]], 
    y = y_train[:5000],
    epochs = 1,
    batch_size = 128
)

40/40 [==============================] - 8s 139ms/step - loss: 2.4432 - accuracy: 0.1368


In [14]:
model.layers[2].trainable = True

for i in range(len(model.layers[2].weights))[-36:]:
    model.layers[2].weights[i]._trainable = True

optimizer = keras.optimizers.Adam(learning_rate=3e-5)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = optimizer, 
    metrics = ['accuracy']
)

model.summary()

Model: "Bert_Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 30)]         0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
tf_bert_for_sequence_classifica TFSequenceClassifier 118302727   input_ids[0][0]                  
                                                                 attention_masks[0][0]            
__________________________________________________________________________________________________
dropout_0 (Dropout)             (None, 7)            0           tf_bert_for_seq

In [15]:
early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 5
)
model_check_point = ModelCheckpoint(
    filepath = f'../model/bert_kor_base_for_classification_fine_tuning_{ver}.h5',
    monitor = 'val_accuracy',
    verbose = 1,
    min_delta = 0.02,
    mode = 'max',
    save_best_only = True,
    save_weights_only = True
)
def scheduler(epoch, lr):
    return lr * 0.9
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(
    x = [X_train_encoded['input_ids'], X_train_encoded['attention_mask']], 
    y = y_train,
    epochs = 30,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [early_stop, model_check_point, lr_scheduler]
)

Epoch 1/30


286/286 [==============================] - 68s 225ms/step - loss: 1.2335 - accuracy: 0.5961 - val_loss: 0.7194 - val_accuracy: 0.8069

Epoch 00001: val_accuracy improved from -inf to 0.80692, saving model to ../model/bert_kor_base_for_classification_fine_tuning_2.h5
Epoch 2/30
286/286 [==============================] - 61s 213ms/step - loss: 0.8555 - accuracy: 0.7810 - val_loss: 0.6934 - val_accuracy: 0.8233

Epoch 00002: val_accuracy improved from 0.80692 to 0.82335, saving model to ../model/bert_kor_base_for_classification_fine_tuning_2.h5
Epoch 3/30
286/286 [==============================] - 61s 212ms/step - loss: 0.8035 - accuracy: 0.8006 - val_loss: 0.6608 - val_accuracy: 0.8348

Epoch 00003: val_accuracy improved from 0.82335 to 0.83485, saving model to ../model/bert_kor_base_for_classification_fine_tuning_2.h5
Epoch 4/30
286/286 [==============================] - 61s 212ms/step - loss: 0.7725 - accuracy: 0.8121 - val_loss: 0.6771 - val_accuracy: 0.8313

Epoch 00004: val_accuracy

In [16]:
best_model = set_model()
best_model.load_weights(f'../model/bert_kor_base_for_classification_fine_tuning_{ver}.h5')

In [17]:
y_train_pred = best_model.predict([X_train_encoded['input_ids'], X_train_encoded['attention_mask']])
y_train_pred = np.argmax(y_train_pred, axis=1)

print(f'train accuracy: {accuracy_score(y_train, y_train_pred):.04f}')

train accuracy: 0.8989


In [18]:
X_test = test['title'].apply(cleaning_text)
X_test = X_test.values.tolist()

X_test_encoded = tokenizer(
    X_test,
    padding = 'max_length',
    truncation = True,
    max_length = data_config['max_length'],
    return_tensors = 'tf'
)

In [19]:
y_test_pred = best_model.predict([X_test_encoded['input_ids'], X_test_encoded['attention_mask']])
y_test_pred = np.argmax(y_test_pred, axis=1)

sample_submission['topic_idx'] = y_test_pred

In [20]:
# sample_submission.to_csv(f'../submit/bert_kor_base_for_classification_{ver}.csv', index=False)

In [21]:
sample_submission.to_csv(f'../submit/bert_kor_base_for_classification_{ver}.csv', index=False)

In [22]:
ver += 1

In [29]:
early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 15
)
model_check_point = ModelCheckpoint(
    filepath = f'../model/bert_kor_base_for_classification_fine_tuning_{ver}.h5',
    monitor = 'val_loss',
    verbose = 1,
    min_delta = 0.02,
    save_best_only = True,
    save_weights_only = True
)
optimizer = keras.optimizers.Adam(learning_rate=3e-5*0.1)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = optimizer, 
    metrics = ['accuracy']
)

def scheduler(epoch, lr):
    if epoch % 3 == 0:
        return lr * 0.7
    else:
        return lr
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit(
    x = [X_train_encoded['input_ids'], X_train_encoded['attention_mask']], 
    y = y_train,
    epochs = 30,
    batch_size = 128,
    validation_split = 0.2,
    callbacks = [early_stop, model_check_point, lr_scheduler]
)

Epoch 1/30


286/286 [==============================] - 66s 216ms/step - loss: 0.5750 - accuracy: 0.8746 - val_loss: 0.5732 - val_accuracy: 0.8477

Epoch 00001: val_loss improved from inf to 0.57321, saving model to ../model/bert_kor_base_for_classification_fine_tuning_3.h5
Epoch 2/30
286/286 [==============================] - 60s 211ms/step - loss: 0.5698 - accuracy: 0.8790 - val_loss: 0.5672 - val_accuracy: 0.8516

Epoch 00002: val_loss improved from 0.57321 to 0.56716, saving model to ../model/bert_kor_base_for_classification_fine_tuning_3.h5
Epoch 3/30
286/286 [==============================] - 60s 211ms/step - loss: 0.5648 - accuracy: 0.8807 - val_loss: 0.5646 - val_accuracy: 0.8512

Epoch 00003: val_loss improved from 0.56716 to 0.56461, saving model to ../model/bert_kor_base_for_classification_fine_tuning_3.h5
Epoch 4/30
286/286 [==============================] - 60s 212ms/step - loss: 0.5623 - accuracy: 0.8800 - val_loss: 0.5688 - val_accuracy: 0.8495

Epoch 00004: val_loss did not improve 

In [30]:
best_model = set_model()
best_model.load_weights(f'../model/bert_kor_base_for_classification_fine_tuning_{ver}.h5')

In [31]:
y_test_pred = best_model.predict([X_test_encoded['input_ids'], X_test_encoded['attention_mask']])
y_test_pred = np.argmax(y_test_pred, axis=1)

sample_submission['topic_idx'] = y_test_pred

In [32]:
sample_submission.to_csv(f'../submit/bert_kor_base_for_classification_{ver}.csv', index=False)